In [71]:
import pandas as pd
import copy

In [9]:
file_directory = "/Users/stanforduser/Desktop/mimic-iii/1.4"

In [70]:
notes_path = f"{file_directory}/NOTEEVENTS.csv"
notes_df = pd.read_csv(notes_path, nrows=1600000)
notes_df = notes_df.loc[(notes_df["DESCRIPTION"] == "Report") & (notes_df["CATEGORY"] == "Nursing/other")]
notes_df["CHARTTIME"] = pd.to_datetime(notes_df["CHARTTIME"])
notes_df

/var/folders/7r/tkpjqnkx5y77zf9bvnb93sg80000gq/T/ipykernel_52181/54097928.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes_df = pd.read_csv(notes_path, nrows=1600000)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1240869,1272898,863,194975.0,2117-10-23,2117-10-23 17:42:00,2117-10-23 17:52:00,Nursing/other,Report,20675.0,NaN,CCU NPN: PLEASE SEE FLOWSHEET FOR OBJECTIVE DA...
1240870,1272899,863,194975.0,2117-10-24,2117-10-24 05:10:00,2117-10-24 05:26:00,Nursing/other,Report,18654.0,NaN,"CCU/MICU NPN 7P-7A\nS:""My shoulder is botherin..."
1240871,1273864,924,111034.0,2126-12-25,2126-12-25 17:08:00,2126-12-25 17:31:00,Nursing/other,Report,15916.0,NaN,CSRU NPN\n\nNeuro: Propofol weaned to 10 mcg/...
1240872,1273865,924,111034.0,2126-12-25,2126-12-25 17:20:00,2126-12-25 17:27:00,Nursing/other,Report,19771.0,NaN,"RESPIRATORY NOTE:\nSEDATION IS BEING WEANED, C..."
1240873,1273866,924,111034.0,2126-12-26,2126-12-26 04:16:00,2126-12-26 04:32:00,Nursing/other,Report,20971.0,NaN,Neuro~ Pt. sedated with Propofol with prn vers...
...,...,...,...,...,...,...,...,...,...,...,...
1599995,1612336,26724,159090.0,2175-04-11,2175-04-11 17:47:00,2175-04-11 17:49:00,Nursing/other,Report,15019.0,NaN,Respiratory care\nPt weaned on vent to psv/cpa...
1599996,1613314,26828,194307.0,2112-08-23,2112-08-23 05:12:00,2112-08-23 05:32:00,Nursing/other,Report,14151.0,NaN,ASSESSEMNT AS NOTED IN CAREVUE\n\nRES: PEEP UP...
1599997,1613315,26828,194307.0,2112-08-23,2112-08-23 18:27:00,2112-08-23 18:30:00,Nursing/other,Report,18795.0,NaN,Resp. Care Note\nPt remains intubated and vent...
1599998,1613316,26828,194307.0,2112-08-23,2112-08-23 19:14:00,2112-08-23 19:24:00,Nursing/other,Report,19263.0,NaN,status\nD: SEDATED ON FENT & VERSED GTT'S..ELE...


In [40]:
diagnoses_path = f"{file_directory}/DIAGNOSES_ICD.csv"
diagnoses_df = pd.read_csv(diagnoses_path)
diagnoses_df

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [78]:
icd_dict_path = f"{file_directory}/D_ICD_DIAGNOSES.csv"
icd_dict_df = pd.read_csv(icd_dict_path)
icd_dict_df = icd_dict_df.set_index("ICD9_CODE")
icd_dict_df

,ROW_ID,SHORT_TITLE,LONG_TITLE
ICD9_CODE,,,
01166,174,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
01170,175,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
01171,176,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
01172,177,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
01173,178,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...
V7399,14432,Scrn unspcf viral dis,Special screening examination for unspecified ...
V740,14433,Screening for cholera,Screening examination for cholera
V741,14434,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis


In [41]:
admissions_path = f"{file_directory}/ADMISSIONS.csv"
admissions_df = pd.read_csv(admissions_path)
admissions_df["ADMITTIME"] = pd.to_datetime(admissions_df["ADMITTIME"])
admissions_df


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [45]:
subjects = set(admissions_df["SUBJECT_ID"])

print("Total people:", len(subjects))

Total people: 46520


In [90]:
datapoints = []
for idx, subject in enumerate(subjects):
    subj_admissions = admissions_df.loc[admissions_df["SUBJECT_ID"] == subject]
    subj_admissions = subj_admissions.sort_values(by="ADMITTIME")
    print("Associated subj_admissinos")
    display(subj_admissions)
    
    history_so_far = []
    for i, row in subj_admissions.iterrows():
        current_hadm_id = row["HADM_ID"]
        print("Examinng visit", current_hadm_id)

        admission_diagonses = diagnoses_df.loc[diagnoses_df["HADM_ID"] == current_hadm_id]
        admission_diagnoses_codes = admission_diagonses["ICD9_CODE"]
        history_so_far.append(list(admission_diagnoses_codes))
        print("History so far", history_so_far)

        notes_from_this_visit = notes_df.loc[notes_df["HADM_ID"] == current_hadm_id]
        print("Notes from visit")
        display(notes_from_this_visit)
        if len(notes_from_this_visit.index) > 0:
            notes_from_this_visit = notes_from_this_visit.sort_values(by="CHARTTIME")

            combined_notes = "\n".join(notes_from_this_visit["TEXT"])
            data_point = (copy.deepcopy(history_so_far), combined_notes)
            datapoints.append(data_point)

    print("********\n\n")
    if idx > 10:
        break

Associated subj_admissinos


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
211,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1


Examinng visit 163353
History so far [['V3001', 'V053', 'V290']]
Notes from visit


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT


********


Associated subj_admissinos


...

********


Associated subj_admissinos


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
222,12,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicaid,NaN,OTHER,NaN,WHITE,NaN,NaN,CORONARY ARTERY DISEASE,0,1


Examinng visit 143045
History so far [['41401', '4111', '25000', '4019', '2720']]
Notes from visit


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1297725,1260766,13,143045.0,2167-01-11,2167-01-11 18:37:00,2167-01-11 18:54:00,Nursing/other,Report,14988.0,NaN,"ALTERED CARDIAC STATUS\nS: ""I HAVE TO USE THAT..."
1297726,1260767,13,143045.0,2167-01-12,2167-01-12 05:56:00,2167-01-12 06:03:00,Nursing/other,Report,20971.0,NaN,Pt. had a comfortable night. Medicated prn wit...
1299271,1260761,13,143045.0,2167-01-09,2167-01-09 11:05:00,2167-01-09 11:18:00,Nursing/other,Report,14990.0,NaN,"ccu npn\ns- having cpain left, l shoulder, arm..."
1299272,1260762,13,143045.0,2167-01-09,2167-01-09 18:40:00,2167-01-09 18:51:00,Nursing/other,Report,20670.0,NaN,Admission to CSRU\n39 y.o. admitted to CSRU af...
1299466,1260763,13,143045.0,2167-01-10,2167-01-10 04:27:00,2167-01-10 04:40:00,Nursing/other,Report,20403.0,NaN,[**2167-1-10**] A:\n\nNeuro: alert and oriente...
1299668,1260764,13,143045.0,2167-01-10,2167-01-10 18:33:00,2167-01-10 18:44:00,Nursing/other,Report,20670.0,NaN,"7a-7p\nCV: NST, no ectopy. On nipride gtt, s..."
1299669,1260765,13,143045.0,2167-01-11,2167-01-11 05:22:00,2167-01-11 05:55:00,Nursing/other,Report,20971.0,NaN,Pt. alert when awakened. Sleeping in long naps...
1300967,1260760,13,143045.0,2167-01-09,2167-01-09 06:02:00,2167-01-09 06:17:00,Nursing/other,Report,16866.0,NaN,CCU ADMISSION NPN 1900-0700: LEFT MAIN\nHPI: I...


********




In [91]:
'''

For subject in SUBJECTS_FROM_ADMISSIONS_TABLE:
    find all of their admissions. (query admissions table)
    order admissions by time

    for admission in time order:
        look up diagnoses_codes for that admission (QUERY diagnoses_ICD table)

        add diagnoses_codes to history_so_far

        look to see if there is a note at that time    (query notes table)
        if note exist:
            generate datapoint of (history_so_far, note)

        continue
        
        

'''

'\n\nFor subject in SUBJECTS_FROM_ADMISSIONS_TABLE:\n    find all of their admissions. (query admissions table)\n    order admissions by time\n\n    for admission in time order:\n        look up diagnoses_codes for that admission (QUERY diagnoses_ICD table)\n\n        add diagnoses_codes to history_so_far\n\n        look to see if there is a note at that time    (query notes table)\n        if note exist:\n            generate datapoint of (history_so_far, note)\n\n        continue\n        \n        \n\n'

In [92]:
renamed_datapoints = []
for datapoint in datapoints:
    history_so_far = datapoint[0]
    combined_notes = datapoint[1]

    renamed_history_so_far = []
    for visit in history_so_far:
        renamed_visit = []
        for code in visit:
            code_name = icd_dict_df.loc[code]["LONG_TITLE"]
            renamed_visit.append(code_name)
        renamed_history_so_far.append(renamed_visit)

    renamed_datapoints.append((renamed_history_so_far, combined_notes))

KeyError: '4440'

In [93]:
datapoints

[([['0389', '78559', '5849', '4275', '41071', '4280', '6826', '4254', '2639']],
  'Micu Progress Nursing Note:\n\nPatient arrived in unit at 19:15 from ED. Hx of 2 days of hypotension in [**Hospital 1**] Rehab, systolic bps in 80s. Foley had been placed the night before admission, with 100cc total urine output of dark, sludgy urine. Patient suffered v-tach cardiac arrest within 30 minutes of arrival in unit, with deterioration to v-fib. Shocked x 2 with 360 j. Dopa, Epi, Neo, Levophed, Lido gtts. Given Adenosine 6, 12,12 after return to v-tach, with synchronized cardioversion after. Also received Fentanyl, total of 200mcg, Versed 2mg, Ativan 4 mg. A-line, right subclavian central line placed. Swan line also placed. Patient has existing Picc line that needs to be dc\'d today with tip sent for culture. Patient received 5000cc NS in unit, with a reported 6 liters given in ED.\nPatient was made a DNR after conference with family. Additionally, blood cultures from 2 sites were sent x two, w

In [89]:
icd_dict_df

,ROW_ID,SHORT_TITLE,LONG_TITLE
ICD9_CODE,,,
01166,174,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
01170,175,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
01171,176,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
01172,177,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
01173,178,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...
V7399,14432,Scrn unspcf viral dis,Special screening examination for unspecified ...
V740,14433,Screening for cholera,Screening examination for cholera
V741,14434,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
